In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [5]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [6]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [7]:
print("% of women who survived:", rate_women)
print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


In [8]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest,chi2
import warnings
warnings.filterwarnings('ignore')

In [9]:
train_data.isnull().mean()*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [10]:
test_data.isnull().mean()*100

PassengerId     0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            20.574163
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.239234
Cabin          78.229665
Embarked        0.000000
dtype: float64

In [11]:
train_data.drop("Cabin", axis = 1, inplace=True)
test_data.drop("Cabin", axis = 1, inplace=True)

In [12]:
si = SimpleImputer()
oe = OrdinalEncoder(handle_unknown="ignore")
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

In [13]:
train_data["family"] = train_data["SibSp"]+train_data["Parch"]+1
test_data["family"] = test_data["SibSp"]+test_data["Parch"]+1


In [14]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,1


In [15]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,3


In [16]:
train_data.drop(["SibSp","Parch"],axis = 1, inplace=True)
test_data.drop(["SibSp","Parch"],axis = 1, inplace=True)


In [17]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,family
535,536,1,2,"Hart, Miss. Eva Miriam",female,7.0,F.C.C. 13529,26.2500,S,3
192,193,1,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,350046,7.8542,S,2
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,3101281,7.9250,S,7
387,388,1,2,"Buss, Miss. Kate",female,36.0,27849,13.0000,S,1
209,210,1,1,"Blank, Mr. Henry",male,40.0,112277,31.0000,C,1


In [18]:
train_data.dropna( subset='Embarked',inplace=True)
test_data.dropna( subset='Embarked',inplace=True)

In [19]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
Ticket           0
Fare             0
Embarked         0
family           0
dtype: int64

In [20]:
train_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)
test_data.drop(["Name","Ticket","Fare"],axis = 1, inplace=True)

In [21]:
train_data.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,Embarked,family
372,373,0,3,male,19.0,S,1
32,33,1,3,female,NaN,Q,1
349,350,0,3,male,42.0,S,1
809,810,1,1,female,33.0,S,2
497,498,0,3,male,NaN,S,1


In [22]:
test_data.head()

,PassengerId,Pclass,Sex,Age,Embarked,family
0,892,3,male,34.5,Q,1
1,893,3,female,47.0,S,2
2,894,2,male,62.0,Q,1
3,895,3,male,27.0,S,1
4,896,3,female,22.0,S,3


In [23]:
X_train = train_data.drop(["Survived","PassengerId"],axis = 1)
y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId",axis = 1)


In [24]:
trf = ColumnTransformer(transformers=[("age_imputation",si,["Age"]),
                                      ("Sex",ohe,["Sex","Embarked"])],remainder="passthrough")

In [25]:
trf1 = RandomForestClassifier()

In [26]:
trf2 = SelectKBest(score_func=chi2, k=5)

In [27]:
pipe = Pipeline(steps=[("trf",trf),("trf2",trf2)
               ,("trf1",trf1)])

In [28]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputation',
                                                  SimpleImputer(), ['Age']),
                                                 ('Sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('trf2',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7ef45bb21ab0>)),
                ('trf1', RandomForestClassifier())])

In [29]:
trf1.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [30]:
params = {"trf1__criterion":["entropy","gini","log_loss"],"trf1__max_depth":[1,2,3,4,5]
         ,"trf1__n_estimators":[50,100,150,200,250]
         
         }

In [31]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'trf', 'trf2', 'trf1', 'trf__n_jobs', 'trf__remainder', 'trf__sparse_threshold', 'trf__transformer_weights', 'trf__transformers', 'trf__verbose', 'trf__verbose_feature_names_out', 'trf__age_imputation', 'trf__Sex', 'trf__age_imputation__add_indicator', 'trf__age_imputation__copy', 'trf__age_imputation__fill_value', 'trf__age_imputation__keep_empty_features', 'trf__age_imputation__missing_values', 'trf__age_imputation__strategy', 'trf__age_imputation__verbose', 'trf__Sex__categories', 'trf__Sex__drop', 'trf__Sex__dtype', 'trf__Sex__handle_unknown', 'trf__Sex__max_categories', 'trf__Sex__min_frequency', 'trf__Sex__sparse', 'trf__Sex__sparse_output', 'trf2__k', 'trf2__score_func', 'trf1__bootstrap', 'trf1__ccp_alpha', 'trf1__class_weight', 'trf1__criterion', 'trf1__max_depth', 'trf1__max_features', 'trf1__max_leaf_nodes', 'trf1__max_samples', 'trf1__min_impurity_decrease', 'trf1__min_samples_leaf', 'trf1__min_samples_split', 'trf1__min_weight_fract

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
grid = GridSearchCV(pipe,params,cv =  5,scoring = "accuracy")

In [34]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('age_imputation',
                                                                         SimpleImputer(),
                                                                         ['Age']),
                                                                        ('Sex',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('trf2',
                                        SelectKBest(k=5,
                                                    score_func=<function chi2 at 0x7ef45bb21ab0>)),
                                       ('trf1', RandomForestClassifier())]),
             param_grid={'trf1__criterion': ['entropy', 'gini', 'log_loss'],
                         'trf1__max_depth': [1, 2, 3, 4, 5],
                         'trf1__n_estimators': [50, 100, 150, 200, 250]},
             scoring='accuracy')

In [35]:
prediction = grid.predict(X_test)

In [36]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
